# Proyecto ACutStic
---

## Objetivo

Programar una herramienta para Adobe Premiere que permita automatizar el proceso de edición de audio. La herramienta deberá seleccionar y cortar trozos de audio similares automáticamente.

## Dot CSV

Proyecto realizado en el contexto #100HorasDeML de dotCSV y resubido en el canal secundario Not CSV.

- [Twich](https://www.twitch.tv/dotcsv)
- [Youtube - Canal principal](https://www.youtube.com/channel/UCy5znSnfMsDwaLlROnZ7Qbg)
- [Youtube - Canal secundario](https://www.youtube.com/c/NotCSV)
- [Twitter](https://twitter.com/DotCSV)

## Dependencias

Para este proyecto se utilizará:

- [Pymiere](https://github.com/qmasingarbe/pymiere)
- [Librosa](https://librosa.org/doc/latest/index.html)
- [Moviepy](https://pypi.org/project/moviepy/)






---
## Inicialización Pymiere

- Obtención de la secuencia activa
- Listado de videos de la secuencia
- Obtención de datos de la secuencia (Ej. FPS)
- Apertura de un video desde el FileSystem hacia Premiere


In [ ]:
import pymiere

from pymiere import wrappers

# Checkear si existe un proyecto abierto
project_opened, sequence_active = wrappers.check_active_sequence(crash=False)
if not project_opened:
    raise ValueError("please open a project")

project = pymiere.objects.app.project

# Abrir secuencias en la Interface si no hay ninguna activa
if not sequence_active:
    sequences = wrappers.list_sequences()
    for seq in sequences:
        project.openSequence(sequenceID=seq.sequenceID)
    project.activeSequence = sequences[0]  # set the first sequence in the list as active

# Listar todos los videos en la secuencia activa
clips = wrappers.list_video(project.activeSequence)

# Obtener los FPS de la secuencia get sequence fps
fps = 1/(float(project.activeSequence.timebase)/wrappers.TICKS_PER_SECONDS)

print("Sequence as a framerate of {} fps".format(fps))

In [ ]:
pathFile = r'C:\Users\Carlos\Twitch\Laboratorio\01.Proyecto ACutStic\input-navidad.mp4'

# Abrimos en el proyecto el archivo de video deseado.
pymiere.objects.app.sourceMonitor.openFilePath(pathFile)

## Extracción del track de audio

In [ ]:
from moviepy.video.io.VideoFileClip import VideoFileClip

videoclip = VideoFileClip(pathFile) 
audioclip = videoclip.audio

In [ ]:
audioclip.write_audiofile('./output/audiofile.wav')

---
## Librosa

- Obtención de la onda de audio.
- Obtención del Sample Rate del audio.
- Conversión de unidad Amplitud -> dB


In [ ]:
import librosa
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
audio_waveform_amplitude, sample_rate = lb.load(pathFile, sr=None)

In [ ]:
audio_waveform = lb.amplitude_to_db(audio_waveform_amplitude)

## Selección de puntos de corte

- Establecimiento del tamaño de ventana de procesamiento
- Establecimiendo de un umbral
- Establecimiendo de un cooldown
- Elección de puntos de corte

In [ ]:
window_size     = int(sample_rate / 2)
threshold       = 60
cooldown        = 5
cooldown_count  = cooldown
cut_points      = np.array([0]) # Añadido inpoint inicial.
is_lower        = False 

for i in np.arange(0, sy.shape[0], window_size):
    
    # Seleccionamos los datos dentro de la ventana.
    audio_window = audio_waveform[i:(window_size + i)]


    # Si estamos por debajo del umbral, y veníamos de arriba...
    win_est = np.abs(np.mean(audio_window))
        
    if win_est <= threshold:
        
        if not is_lower: 
            # Guardamos el sample rate.
            cut_points = np.append(cut_points, i)
            is_lower = True
            cooldown_count = cooldown
            
    elif is_lower and cooldown_count <= 0:
        is_lower = False
        cut_points = np.append(cut_points, i)
        cooldown_count = cooldown
        
    cooldown_count -= 1

print('Puntos de corte: {}'.format(len(cut_points)))

## Visualización de puntos de corte

In [ ]:
from librosa.display import waveplot

plt.figure(figsize=(20, 10))    
lb.display.waveplot(audio_waveform_amplitude, sample_rate)
plt.scatter(cut_points / sample_rate, np.zeros(len(cut_points)), c="red")
plt.show()


plt.figure(figsize=(20, 10))    
# plt.scatter(cut_points, np.zeros(len(cut_points)), c="red")
plt.plot(audio_waveform)
plt.show()

---
## Inserción de clips cortados

- Extracción de puntos de corte
- Inserción en tracks separados
- Etiquetado/Coloreado de clips


In [ ]:
from pymiere.core import PymiereBaseObject, PymiereBaseCollection, Array, _format_object_to_py, _format_object_to_es

project_vid = project.rootItem.children[-1]

In [ ]:
def setColorLabel(clip, label): 
    clip._eval_on_this_object("setColorLabel({})".format(_format_object_to_es(label)))

def getColorLabel(position):
    return 8 + abs(1 - position % 2) * 2

def getTrack(position):
    return (position % 2) * 2

In [ ]:
timestamp = 0
clip = project.rootItem.children[-1]

to_remove = True

for i in range(len(cut_points) - 1):
    
        in_point = cut_points[i]    / sample_rate
        out_point = cut_points[i+1] / sample_rate

        print('InPoint: {} - OutPoint: {}'.format(in_point, out_point))

        clip.setInPoint(in_point,  4)
        clip.setOutPoint(out_point, 4)
        
        setColorLabel(clip, getColorLabel(i))
        project.activeSequence.videoTracks[getTrack(i)].insertClip(clip, timestamp)

        timestamp += (out_point - in_point)